In [1]:
pip install cloudant

Note: you may need to restart the kernel to use updated packages.


In [2]:
from cloudant.client import Cloudant

CLOUDANT_USERNAME = "5be7a40b-e332-483e-b357-1cf5685ed228-bluemix"
CLOUDANT_API_KEY = "pWRQLruHj79Xp6DSpEldftTtgdNBH3-7MAva2xtQoIao"  # Use your API key here
DATABASE_NAME = "retail_transaction"

# Connect to Cloudant
client = Cloudant.iam(CLOUDANT_USERNAME, CLOUDANT_API_KEY, connect=True)
db = client[DATABASE_NAME]

print("Connected to Cloudant database:", DATABASE_NAME)


Connected to Cloudant database: retail_transaction


In [4]:
# Replace with your IBM Cloudant credentials
CLOUDANT_USERNAME = "5be7a40b-e332-483e-b357-1cf5685ed228-bluemix"
CLOUDANT_API_KEY = "pWRQLruHj79Xp6DSpEldftTtgdNBH3-7MAva2xtQoIao"
DATABASE_NAME = "retail_transaction"
CLOUDANT_SERVICE_URL="https://5be7a40b-e332-483e-b357-1cf5685ed228-bluemix.cloudantnosqldb.appdomain.cloud"

# Connect to Cloudant
client = Cloudant.iam(CLOUDANT_USERNAME, CLOUDANT_API_KEY, connect=True)
db = client[DATABASE_NAME]

# Print all document IDs in the database
print([doc['_id'] for doc in db])


['058f0643cabe09ffc7500206dc010be5', '058f0643cabe09ffc7500206dc013ffe', '058f0643cabe09ffc7500206dc015587', '058f0643cabe09ffc7500206dc0163c9', '058f0643cabe09ffc7500206dc0163de', '058f0643cabe09ffc7500206dc016fd4', '156adc19966956efb8fe4b2fd5019d6c', '156adc19966956efb8fe4b2fd501a86b', '156adc19966956efb8fe4b2fd501a8f9', '156adc19966956efb8fe4b2fd501f761', '156adc19966956efb8fe4b2fd50213ea', '70a4507892623d5b3ec435da0f013859', '70a4507892623d5b3ec435da0f018657', '70a4507892623d5b3ec435da0f018c86', '70a4507892623d5b3ec435da0f01a95f', 'a85c0316c27e1897816fb9df0501351f', 'a85c0316c27e1897816fb9df05014689', 'a85c0316c27e1897816fb9df05018e87', 'a85c0316c27e1897816fb9df05019ec0', 'da57a757c7af6e470531d3648401a520', 'da57a757c7af6e470531d3648401b297', 'da57a757c7af6e470531d3648401bcd2', 'da57a757c7af6e470531d3648401c592', 'da57a757c7af6e470531d364840212b0', 'db4bef538ffc0d1f25d982e86701428b', 'db4bef538ffc0d1f25d982e867016fd7', 'db4bef538ffc0d1f25d982e867017462', 'db4bef538ffc0d1f25d982e867

In [7]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import random

# Sample transaction data from my dataset
data = pd.DataFrame({
    'CustomerID': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    'ProductID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Quantity': [1, 5, 2, 10, 5000, 10000, 500, 100, 50, 20000],  # Extreme values for anomalies
    'Price': [20, 50, 15, 100, 200, 1, 999, 75, 0.5, 5000],  # Some extreme values
    'TransactionDate': pd.date_range(start='2025-02-01', periods=10, freq='D'),
    'PaymentMethod': ['Credit Card', 'Debit Card', 'Cash', 'UPI', 'Credit Card', 'Crypto', 'Cash', 'UPI', 'Debit Card', 'Crypto'],
    'StoreLocation': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Miami', 'San Francisco', 'Boston', 'Seattle', 'Denver', 'Austin'],
    'ProductCategory': ['Electronics', 'Clothing', 'Groceries', 'Appliances', 'Furniture', 'Toys', 'Sports', 'Beauty', 'Books', 'Jewelry'],
    'DiscountApplied(%)': [5, 10, 0, 50, 5, 99, 1, 100, 25, 75],  # Some high discounts for anomalies
    'TotalAmount': [19, 45, 15, 90, 100000, 0.5, 9999, 7500, 0.25, 5000000]  # Some extreme anomalies
})

# Calculate expected TotalAmount if not already provided
data['CalculatedTotalAmount'] = data['Quantity'] * data['Price'] * (1 - data['DiscountApplied(%)'] / 100)

# Select numerical features for anomaly detection
features = ['Quantity', 'Price', 'DiscountApplied(%)', 'CalculatedTotalAmount']

# Scale features
scaler = MinMaxScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data[features]), columns=features)

# Apply Isolation Forest (adjust contamination to detect at least 4 anomalies)
model = IsolationForest(contamination=0.4, random_state=42)
data['anomaly'] = model.fit_predict(data_scaled)

# Function to generate alerts for detected anomalies
def generate_alerts(df):
    anomalies = df[df['anomaly'] == -1]
    if anomalies.empty:
        print("✅ No anomalies detected.")
    else:
        print("\n🚨 ALERTS: Potential Data Integrity Issues Detected 🚨")
        for index, row in anomalies.iterrows():
            print(f"⚠ CustomerID {row['CustomerID']} at {row['StoreLocation']} made a suspicious transaction.")
            print(f"   - Product: {row['ProductCategory']} (ID: {row['ProductID']})")
            print(f"   - Quantity: {row['Quantity']}, Price: ${row['Price']}")
            print(f"   - Discount Applied: {row['DiscountApplied(%)']}%")
            print(f"   - TotalAmount: ${row['TotalAmount']}")
            print(f"   - Payment Method: {row['PaymentMethod']}")
            print("-" * 50)

# Generate alerts for anomalies
generate_alerts(data)

# Display detected anomalies
print("\n🔍 Detected Anomalies:")
print(data[data['anomaly'] == -1])


🚨 ALERTS: Potential Data Integrity Issues Detected 🚨
⚠ CustomerID 105 at Miami made a suspicious transaction.
   - Product: Furniture (ID: 5)
   - Quantity: 5000, Price: $200.0
   - Discount Applied: 5%
   - TotalAmount: $100000.0
   - Payment Method: Credit Card
--------------------------------------------------
⚠ CustomerID 106 at San Francisco made a suspicious transaction.
   - Product: Toys (ID: 6)
   - Quantity: 10000, Price: $1.0
   - Discount Applied: 99%
   - TotalAmount: $0.5
   - Payment Method: Crypto
--------------------------------------------------
⚠ CustomerID 107 at Boston made a suspicious transaction.
   - Product: Sports (ID: 7)
   - Quantity: 500, Price: $999.0
   - Discount Applied: 1%
   - TotalAmount: $9999.0
   - Payment Method: Cash
--------------------------------------------------
⚠ CustomerID 110 at Austin made a suspicious transaction.
   - Product: Jewelry (ID: 10)
   - Quantity: 20000, Price: $5000.0
   - Discount Applied: 75%
   - TotalAmount: $5000000